In [0]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
!pip install nltk

import itertools
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

# Fonction permettant de rassembler les n mots les plus fréquents d'un fichier texte dans un seul dataframe

def mostcommonwords(path,n):
    file_location = path
    rdd = sc.textFile(file_location)
    # On récupère le contenu des fichiers texte
    line = rdd.take
    lines = rdd.collect()
    words = []
    # On sépare le texte en lignes
    for stuff in lines:
        words.append(stuff.split())
    
    words = list(itertools.chain.from_iterable(words))
    # On sépare les lignes en mots pour ensuite créer le dataframe qui contiendra les mots et leur nombre d'occurences

    dict = {'word':[],
            'occurences':[]}
    df_occurences = pd.DataFrame(dict)

    # Tout d'abord, on assigne à tous les mots une occurence de 1
    for word in words:
        df_occurences.loc[len(df_occurences.index)] = [word, 1]

    # Par la suite, on retire tous les symboles de ponctuation dans le dataframe 
    for thing in [*string.punctuation]:
        df_occurences.drop(df_occurences[df_occurences['word']==thing].index, inplace = True)

    # Enfin, on supprime les mots de liaison tels que 'the', 'an' ou encore 'and'. 
    # Pour l'instant, nous nous sommes limités aux textes en anglais
    for bip in stopwords.words("english"):
        df_occurences.drop(df_occurences[df_occurences['word']==bip].index, inplace = True)

    # Pour obtenir le nombre d'occurence de chaque mot unique, on regroupe les mots identiqus 
    # en sommant les occurences puis en les classant par ordre décroissant  
    df_occurences = df_occurences.groupby(['word'],as_index=False).sum().sort_values(by=['occurences'], ascending=False)

    # On ne récupère que les n mots les plus fréquents
    df_occurences_top = df_occurences[:n]

    # Enfin on réétablie l'index dont l'ordre a été perturbé par le shuffle and sort
    df_occurences_top = df_occurences_top.set_index(pd.Index(range(n)))
    
    return df_occurences_top





You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-04c8e94c-3d5e-4c92-a25b-1afc0c7a9a9f/bin/python -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:


# D'abord on assigne aux dataframes un nom correspondant à leur fichier d'origine
def nomenclatura(frames, names):
    i=0
    for df in frames:
        df.attrs['name']=names[i]
        i+=1
    return frames
# On rassemble les noms des dataframes en comités. Les membres de chaque comités sont stockés dans un dictionnaire
def gathering(frames):
    i=1
    d={}
    for x in range(1,len(frames)+1):
        d["Committee {0}".format(x)] = []
    for dfe in frames:
        d['Committee '+str(i)].append(dfe.attrs['name'])
        for dfco in frames:
            comp = pd.merge(dfe,dfco,how='inner',on='word')
            # Si parmi les 10 mots les plus fréquents 3 sont communs aux deux textes alors 
            # on considère que les textes apartiennent au même comité
            if comp.shape[0]<dfco.shape[0] and comp.shape[0]>3:
                d['Committee '+str(i)].append(dfco.attrs['name'])        
        i+=1
    return d



In [0]:
# Une fois les comités établis, on crée des dossiers pour chaque comité où seront stockés les fichiers membres
# Dans chaque dossier on copie les fichiers qui lui sont assignés

def directorycreation(dictionnary,source,destination='/FileStore'):  
    for i in range(1,len(dictionnary)+1):
        dbutils.fs.mkdirs(destination+"/Committee"+str(i))
        for frame in dictionnary['Committee '+str(i)]:            
            dbutils.fs.cp(source+frame+".txt",destination+"/Committee"+str(i)+"/"+frame+".txt")   
        i+=1
    return 0
           

In [0]:
# On applique la fonction à 6 fichiers text contenant des critiques de film en gardant le même nom que celui du texte 

cv994_12270 = mostcommonwords('FileStore/pos/cv994_12270.txt',11)
cv995_21821 = mostcommonwords('FileStore/pos/cv995_21821.txt',11)
cv996_11592 = mostcommonwords('FileStore/pos/cv996_11592.txt',11)
cv997_5046 = mostcommonwords('FileStore/pos/cv997_5046.txt',11)
cv998_14111 = mostcommonwords('FileStore/pos/cv998_14111.txt',11)
cv999_13106 = mostcommonwords('FileStore/pos/cv999_13106.txt',11)

cv997_5046.head(11)

,word,occurences
0,black,15
1,soldiers,13
2,shaw,12
3,glory,7
4,film,7
5,war,7
6,white,7
7,one,5
8,battle,5
9,washington,5


In [0]:
# Une fois les dataframes créés on peut les rassembler selon les similitudes  

dataframes=[cv994_12270,cv995_21821,cv996_11592,cv997_5046,cv998_14111,cv999_13106]
names=['cv994_12270','cv995_21821','cv996_11592','cv997_5046','cv998_14111','cv999_13106']

# On assigne un nom à chaque dataframe
dataframes = nomenclatura(dataframes,names)

# On peut alors former les comités
Committees = gathering(dataframes)

# On place les fichiers dans leurs dossier respectifs 
directorycreation(Committees,source='/FileStore/pos/')  

# On peut afficher ici le dictionnaire contenant la liste des membres de chaque comité
print(Committees)

{'Committee 1': ['cv994_12270', 'cv995_21821'], 'Committee 2': ['cv995_21821', 'cv994_12270'], 'Committee 3': ['cv996_11592'], 'Committee 4': ['cv997_5046', 'cv998_14111'], 'Committee 5': ['cv998_14111', 'cv997_5046'], 'Committee 6': ['cv999_13106']}


In [0]:
%fs
ls /FileStore/Committee2




path,name,size,modificationTime
dbfs:/FileStore/Committee2/cv994_12270.txt,cv994_12270.txt,2992,1715017608000
dbfs:/FileStore/Committee2/cv995_21821.txt,cv995_21821.txt,4241,1715017608000
